In [2]:
import json
import time
import pickle
from functools import reduce

import pandas as pd
import numpy as np
import markovify

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from tqdm.autonotebook import tqdm

from py2neo import Graph, Node, Relationship

In [3]:
reader = pd.read_json('/tmp/data/authorlist_titles.json', lines=True, chunksize=1)

In [4]:
models = {}

for chunk in tqdm(reader):

    aid  =  chunk['a.id'].values[0]
    cleanName = chunk['cleanName'].values[0]
    name = chunk['name'].values[0]
    num_titles = chunk['num_titles'].values[0]
    titles = chunk['titles'].values[0]
    
    if num_titles > 5:
        print("Training model for {}...".format(name), end="")
        corpus = '\n'.join(titles)
        mdl = markovify.NewlineText(corpus, state_size=2, retain_original=False, well_formed=False)
        models[aid] = {'cleanName':cleanName, 'name':name, 'num_titles': num_titles, 'model':mdl}
        print("Done.")

ValueError: Expected object or value

In [141]:
corpus



'wanted: amateur sleuths. (book reviews: meteorites)'

In [125]:
titles

['music with everything',
 'jogging on the spot',
 'daedalus: vitamins and minerals',
 'daedalus: power in the dust',
 'the dark is light enough',
 'blood and tears',
 'daedalus: new smells for old',
 'daedalus: slow rubber',
 'size and shape',
 'daedalus: the light rolls in',
 'daedalus: spinning and leaking',
 'antitermination of amidase expression in pseudomonas aeruginosa is controlled by a novel cytoplasmic amide-binding protein.',
 'sex or violence',
 'washing the wind',
 'daedalus: back to the womb',
 'read my tyres',
 'the highway code',
 'a sideways move',
 'god plays dice',
 'the light metallic',
 'seeing the strain',
 'daedalus: solid explosion',
 'bending the bugs',
 'crme de gaz',
 'sun and air',
 'daedalus: direct illumination',
 'daedalus: learning to forget',
 'daedalus: cleaning the cleaner',
 'clearing the air',
 'daedalus: zero-tolerance policing',
 'water at bay',
 'daedalus: negative friction',
 'adenylate kinase in human tissues i. organ specificity of adenylate k

In [ ]:
import pickle

p={1:2}
q={3:4}
filename="picklefile"
with open(filename, 'a+') as fp:
    pickle.dump(p,fp)
    pickle.dump(q,fp)


#To load from pickle file
data = []
with open(filename, 'rb') as fr:
    try:
        while True:
            data.append(pickle.load(fr))
    except EOFError:
        pass

In [ ]:

models = {}
for author in author_list:
    print("Training model for {}...".format(author))
    authorcorpus = titledf.loc[titledf['author']==author,'title'] \
                    .str.cat(sep="\n")
    
    authormodel = markovify.NewlineText(authorcorpus, state_size=2)
    models[author] = authormodel

In [8]:
public_address = '54.174.175.98'
graph = Graph('bolt://{}:7687'.format(public_address), auth=('neo4j','myneo'))

def run_query(query, graph, print_query=False, run_query=True, 
              print_only=False, to_df=False, verbose=True):
    df = 1
    if print_only: 
        print_query = True
        run_query = False
    start_time = time.time()
    if print_query:
        print(query)
    if run_query:
        if to_df:
            df = graph.run(query).to_data_frame()
        else:
            graph.run(query)
    end_time = time.time()
    minutes_elapsed = (end_time-start_time)/60
    if verbose:
        print("Query completed in {:.2f} minutes.".format(minutes_elapsed))
    return df

In [9]:
author_list = ['jcraigventer',
               'edwardsboyden',
               'davidbotstein',
               'georgemchurch',
               'shuguangzhang',
               'josephmjacobson']

titledf = pd.DataFrame()

for author in author_list:
    query = """
    match (a:Author)-[:AUTHORED]->(q:Quanta)
    where a.cleanName='{}'
    return a.cleanName as author, q.title as title    
    """.format(author)
    authordf = run_query(query, graph, to_df=True)
    titledf = pd.concat([authordf, titledf])

Query completed in 0.00 minutes.
Query completed in 0.00 minutes.
Query completed in 0.00 minutes.
Query completed in 0.00 minutes.
Query completed in 0.00 minutes.
Query completed in 0.00 minutes.


In [10]:
import markovify

models = {}
for author in author_list:
    print("Training model for {}...".format(author))
    authorcorpus = titledf.loc[titledf['author']==author,'title'] \
                    .str.cat(sep="\n")
    
    authormodel = markovify.NewlineText(authorcorpus, state_size=2, retain_original=False)
    models[author] = authormodel

Training model for jcraigventer...
Training model for edwardsboyden...
Training model for davidbotstein...
Training model for georgemchurch...
Training model for shuguangzhang...
Training model for josephmjacobson...


In [11]:
for author in author_list:
    print("\n=== {} ===".format(author))
    for i in range(5):
        print("{}\n".format(models[author].make_sentence(tries=100)))


=== jcraigventer ===
Corrections: The complete genome sequence of the Human Malaria Parasite Plasmodium

The first genome from the rhesus macaque genome.

Enzymatic assembly of the cholera pathogen Vibrio cholerae

Automated DNA sequencing and analysis of 106 kilobases from human chromosome 19q13.3

Synthetic biology: Construction of a minimal bacterium


=== edwardsboyden ===
Optogenetics and the future of neuroscience

Optogenetic astrocyte activation modulates response selectivity of visual cortex neurons in vivo

Modulation of nitrogen vacancy charge state and fluorescence in nanodiamonds using electrochemical potential

Optogenetic astrocyte activation modulates response selectivity of visual cortex neurons in vivo

All-optical electrophysiology in mammalian neurons using engineered microbial rhodopsins


=== davidbotstein ===
ADP-ribosylation factor is functionally and physically associated with the Golgi complex.

A molecular barcoded yeast ORF and intergenic microarray

Genomi

In [174]:
query = """
match (q:Quanta)
return q.title as title    
"""
alltitledf = run_query(query, graph, to_df=True)

print("== Training model for all titles ==")
alltitlecorpus = alltitledf['title'].str.cat(sep="\n")
alltitlemodel = markovify.NewlineText(alltitlecorpus,
                                      retain_original=False,
                                      well_formed=True,
                                      state_size=2)

models['allauthors'] = alltitlemodel

print("== Done. ==\n")
for _ in range(10):
    print("{}\n".format(alltitlemodel.make_sentence(tries=100)))

Query completed in 1.30 minutes.
== Training model for all titles ==
== Done. ==

A New Inexpensive and Easily Prepared Post-Transition Metal Catalysts

Lehrbuch der Tropenkrankheiten

Medical Care: A Survey of the Glass Research at Last

Gut microbiome and enriches for a defective beta-D-galactosidase. II. Immunological relationship between the Basic Rocks

Birth of a Tetranuclear Manganese Aggregate Exhibiting Short OċO Interactions

Precision engineering for a leader.

Regiospecific anodic cyanation of aromatics from D-glucose: rate-limiting enzymes in oxidative stress injury

Methyl-coenzyme M Reductase

Managing the Detail Man-Reply

Light-directed, programmable microarray synthesis



In [15]:
author_weightings = {'jcraigventer':    0,
                     'edwardsboyden':   0,
                     'davidbotstein':   0,
                     'georgemchurch':   5,
                     'shuguangzhang':   0,
                     'josephmjacobson': 100}
#                      'allauthors': 0}

combinedmodel = markovify.combine(
                    [models[a] for a in author_weightings.keys()],
                    list(author_weightings.values()))

In [16]:
for _ in range(200):
    print("{}\n".format(combinedmodel.make_sentence(tries=100)))

Corrigendum: Targeted and genome-scale strategies reveal gene-body methylation signatures in human cells

An electrophoretic ink for all-printed reflective electronic displays

Multiplex single-molecule interaction profiling of DNA sequence variant libraries using CRISPR–Cas9 in yeast metabolism

Clinical assessment incorporating a personal genome

A Logic-Gated Nanorobot for Targeted Transport of Molecular Payloads

Synthesis of monofunctionalized gold nanoparticles by fmoc solid-phase reactions.

Remote electronic control of DNA microarrays.

Synthesis of monofunctionalized gold nanoparticles by fmoc solid-phase reactions.

An integrated strategy for analyzing the unique developmental programs of different myoblast subtypes.

CRISPR–Cas encoding of a Novel Set of Genes Regulated by a Unique Liver X Receptor-α-mediated Transcription Mechanism

Synthesis of monofunctionalized gold nanoparticles by fmoc solid-phase reactions.

Bacteria Subsisting on Antibiotics

Engineering an allosteri